In [1]:
import numpy as np 
import random
from scipy.spatial.transform import Rotation
import sfunc as sf
import matplotlib.pyplot as plt
import os
import sys
%matplotlib qt 


In [2]:
main = os.getcwd()
os.chdir("../")
pathf = os.getcwd()
os.chdir(main)

In [3]:
sys.path.insert(1, pathf)
import sfunc as sf

# Epipolar Geometry

Set up two cameras, both with the internal parameters:

$$ K = \begin{bmatrix} 1000 & 0 & 300 \\ 0 & 1000 & 200 \\ 0 & 0 & 1 \end{bmatrix}$$

Now, for the first camera — let us call that `Cam` — set the rotation to identity $R_1 = I$ and set the translation to zero $t_1 = 0$. For the second camera `Cam` use the rotation given by the R function: $R_2= R(0.7,−0.5,0.8)$.


$$ R(\theta_x,\theta_y,\theta_z) = \begin{bmatrix} cos(\theta_z) & -sin(\theta_z) & 0 \\ sin(\theta_z) & cos(\theta_z) & 0 \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} cos(\theta_y) & 0 & -sin(\theta_y)\\ 0 & 1 & 0 \\ sin(\theta_y) & 0 & cos(\theta_y) \end{bmatrix} \begin{bmatrix} 1 & 0 & 0\\ 0 & cos(\theta_x) & -sin(\theta_x) \\ 0 & sin(\theta_x) & cos(\theta_x) \end{bmatrix}$$

and the translation:
$$ t_2 = \begin{bmatrix} 0.2 \\ 2 \\ 1 \end{bmatrix}$$


In [4]:
K = np.matrix(np.array([[1000, 0, 300], [0, 1000, 200], [0, 0, 1]]))

In [5]:
R1 = np.matrix(np.eye(3))
t1 = np.zeros(3)
R2 = np.matrix(Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix())
t2 = np.array([0.2, 2, 1])

In [6]:
Q = np.array([[1,0.5,4,1]]).T

In [7]:
q1_hom,q1,_ = sf.projectpoints(K, R1, t1, Q)

In [8]:
q2_hom,q2,_ = sf.projectpoints(K,R2, t2, Q)

In [9]:
q1, q2

(matrix([[550.],
         [325.]]),
 matrix([[582.47256835],
         [185.98985776]]))

 ## Cross Operation

$$ CrossOp(p) = [p]_x = \begin{bmatrix} 0 & -z & y \\ z & 0 & -x \\ -y & x & 0  \end{bmatrix}$$

As a good habit, verify that your function works by testing it on random vectors to ensure that:
$$ [p_1]_x p_2 = p_1 x p_2 $$


In [10]:
def crossOp(v):
    """
    v: is a vector, should it be in column vector 
    """
    crossv = np.matrix(np.array([[0, -v[2], v[1]],[v[2], 0, -v[0]], [-v[1], v[0], 0]]))
    return crossv

In [11]:
v = np.array([1,2,3])
crossOp(v)

matrix([[ 0, -3,  2],
        [ 3,  0, -1],
        [-2,  1,  0]])

Chack if they are the same 

In [12]:
v = np.array([1,2,3])
v2 = np.array([7,2,4])
np.cross(v, v2)

array([  2,  17, -12])

In [13]:
v = np.array([1,2,3])
v2 = np.array([[7,2,4]]).T
crossOp(v)*v2

matrix([[  2],
        [ 17],
        [-12]])

# Fundamental Matrix : F

In [14]:
def fundmat(K1, K2, t1, R1):
    """
    K1, K2: Camera Matrices, 
    t1: translation vector camera 1
    R1: Rotation matrix camera 1
    Returns: Fundamental matrix 
    """
    # check the shape of the matrices
    if t1.shape == (3,1):
        pass
    else:
        t1 = t1.T
    
    # Compute the inverses
    K1_inv, K2_inv = np.linalg.inv(K1), np.linalg.inv(K2)
    # Compute essential matrix 
    E = crossOp(t1)@R1
    F = K2_inv.T @ E @ K1_inv
    return F

In [15]:
F = fundmat(K,K,t2,R2)
F

matrix([[ 3.29311881e-07,  8.19396327e-07,  1.79162592e-03],
        [ 5.15532551e-07, -8.76915984e-07,  9.31426656e-05],
        [-1.29882755e-03,  1.51951700e-03, -1.10072682e+00]])

## Epipolar Lines
In order to use the Fundamental Matrix we have to use the homogeneous points.                                                   
                                $l_2 = F*q_1$

In [16]:
l2 = F@q1_hom
l2

matrix([[ 8.95620504e-03],
        [ 3.66751496e-04],
        [-5.28495581e+00]])

#### To check if a point lien in on line we can check it with:
$l^T*q = 0$  
Where $l$ is the line and $q$ the point in homogenous coordinates 

In [17]:
np.round(l2.T*q2_hom)

matrix([[0.]])

Let $Q$ and $Q'$ denote the same 3D point in world space and in the frame of camera one. In other words we have relation:

$$ Q' = \begin{bmatrix} R_1 & t_1 \\ 0 & 1 \end{bmatrix} Q $$

Solve
$$ Q = \begin{bmatrix} R_1^T & -R_1^T t_1 \\ 0 & 1 \end{bmatrix} Q' $$

<font color='darkblue'> 
    
$$ Q' = \begin{bmatrix} R_1 & t_1 \\ 0 & 1 \end{bmatrix}  \begin{bmatrix} R_1^T & -R_1^T t_1 \\ 0 & 1 \end{bmatrix} Q'$$
    
    
$$ Q' = \begin{bmatrix} R_1 R_1^T & -R_1R_1^T+t_1 \\ 0 & 1 \end{bmatrix}Q'$$
    
$$ Q' = \begin{bmatrix} I & 0 \\ 0 & 1 \end{bmatrix}Q'$$
    
And we find that it is valid! This is true as the matrices are inverses of each other.

Show that the can work only in the coordinate system of camera one, by showing that we can project points with


$$ q_1 = K \begin{bmatrix} I & 0 \end{bmatrix} Q' ,   q_2 = K \begin{bmatrix} R_2' & t_2' \end{bmatrix} Q'$$

where

$$ R_2' = R_2 R_1^T, t_2'= t_2 - R_2 R_1^T t_1$$

<font color='darkblue'> For the first projection in camera one we reduce the projection equation: 

<font color='darkblue'>
$$ q_1 = K\begin{bmatrix} R_1| & t_1 \end{bmatrix} Q, $$ 
$$ = K \begin{bmatrix} I & 0 \end{bmatrix} \begin{bmatrix} R_1 & t_1\\ 0 & 1 \end{bmatrix}Q , $$
    
$$ = K \begin{bmatrix} I & 0 \end{bmatrix} Q' , $$

    
<font color='darkblue'> For the second projection into camera two we insert: 

<font color='darkblue'>
$$ q_2 = K\begin{bmatrix} R_2|& t_2 \end{bmatrix} Q, $$ 
$$ q_2 = K \begin{bmatrix} R_2|& t_2 \end{bmatrix} \begin{bmatrix} R_1 & t_1\\ 0 & 1 \end{bmatrix} Q' , $$
$$ q_2 = K \begin{bmatrix} R_2 R_1^T & t_2-R_2't_1 \end{bmatrix} Q' , $$
$$ q_2 = K \begin{bmatrix} R_2' & t_2' \end{bmatrix} Q' . $$
    
where $$ R_2'= R_2R_1^T , t_2' = t_2 - R_2't_1  $$ 

## Applied Epipolar Geometry

In [18]:
import numpy as np 
import random
from scipy.spatial.transform import Rotation
import sfunc as sf
import matplotlib.pyplot as plt

In [19]:
d = np.load('TwoImageData.npy', allow_pickle=True).item()  # Dicctionary
print(d.keys())

dict_keys(['im1', 'im2', 'R1', 'R2', 't1', 't2', 'K'])


In [20]:
im1 = d['im1']
im2 = d['im2']
R1 = d['R1']
R2 = d['R2']
t1 = np.squeeze(d['t1'] )
t2 = np.squeeze(d['t2'])
K = d['K']

In [21]:
F = fundmat(K,K,t2,R2)
F

matrix([[ 6.67972386e-12, -7.85049967e-10,  1.17921973e-07],
        [-9.75936980e-10, -4.86806510e-12,  3.28699196e-05],
        [ 4.23506610e-07, -3.21704080e-05, -2.12002228e-04]])

In [22]:
im1.shape
type(im1)

numpy.ndarray

In [23]:
fig = plt.subplots()
plt.imshow(im1)
n = plt.ginput(1)
plt.show()


In [24]:
n

[]

In [25]:
plt.figure(figsize=(15,5))


<Figure size 1500x500 with 0 Axes>

In [26]:
import os 
os.getcwd()

'C:\\Users\\G531\\Documents\\8 - Github\\ComputerVision\\3- Multiview geometry'

In [27]:
def dis(im1, im2):
    plt.subplots(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.imshow(im1)
    plt.subplot(1,2,2)
    plt.imshow(im2)

In [28]:
dis(im1,im2)

# Triangulation

In [29]:
def squeezdim(v):
    """
    Function that squeezes a vetor into a the form (n,)
    v: vector of dimension n.     
    """
    v = np.array(v)
    try:
        m,n = v.shape
    except ValueError:
        return v
    if m>n: 
        v = np.squeeze(v, axis=1)
    else:
        v = np.squeeze(v, axis=0)
    return v

### Calculate matrix B for 1 point and a P 

In [30]:
def matrix_B_tirnagulation(q, P):
    """
    Creates  B matrix for Triangulation. Follows the next equation:
        qh = P*Q
    Where qh is a 2D point in homoogenous form [sx, sy, s], P is the Projection Matrix and Q it's a 3D point in homogenous
    coordinates [X, Y, Z, 1].
    We can rearange it as:
        s * [x, y].T = [p(1)*Q, p(2)*Q].T
    Where p(i) = row vector of P. P(1) = P[0,:] (1,3) and the scale s = p(3)*Q:
        0 = [p(3)*x-p(1), p(3)*y-p(2)].T*Q
    
    PARAMETERS
    q: Point in homogenous fom [sx, sy, s] -> (3,1)
    P: projection matrix K*[R t] -> (3,4)
    RETURNS
    B: B Matrix (2,4)
    """

    q = squeezdim(q)
    
    if q.shape != (3,):
        print('q is not in Homogenous form: ', q.shape)
        return False
    # Transform into inhomogeous form [x, y, 1] in case it's not in homogenous form. 
    q = q/q[-1]
    
    
    #  Extract parameters 
    x = q[0]
    y = q[1]
    p_1 = squeezdim(P[0,:])
    p_2 = squeezdim(P[1,:])
    p_3 = squeezdim(P[2,:])
    
    # Create B
    B = np.array([(p_3*x - p_1), (p_3*y-p_2)])
    
    return B

In [31]:
def triangulate(qs:list, P:list):
    """
    Find points in 3D given a set of points in 2D and the Cameras' Projection Matrices used to obtain those 2D points.
    Using Linear algorithm.
    Create a Matrix B fof (n*2,4) where n is the number of cameras. Check Triabngulate_B for further doc.
    One We have B we apply singular value decomposition.
        arg min||B*Q||_2
    Wehere we impose the constrain of:
        ||Q||_2 = 1
    To specify that Q hasn't to be 0.
    
    PARAMETERS
    qs: List of points in 2D points in homogenous from. Scale Matters. 
    P: List of Projection Matrix.
    
    RETURNS
    Q: Estimated point in 3D in Homogenous coordinates [X, Y, Z, 1]
    """
    
    # Check The same samount of points and cameras.
    if len(qs) != len(P):
        print('Different number of 2D points and Projection matrix.')
        return False
    
    # Get the number of cameras
    n = len(P)
    # Set step for the B matrix 
    step = 2

    # Initializate B
    B = np.zeros((step*n, 4))
    
    for p,q,i in zip(P,qs, range(0,n*step,step)):
        B[i:i+step] = matrix_B_tirnagulation(q,p)

    _,s,eigVec = np.linalg.svd(B)
    # Find Min 
    idx = np.where(s==min(s))
    # Select the eigenvector correspondent to the eigenvalue
    Q = eigVec[idx]
    
    return  Q

In [32]:
def projectMat(K, R, t):
    """
    Given the Camera Matrix, the Rotation Matrix and the translation vector. Creates the Projection Matrix.
    P = K*[R t]
    
    K-> Camera Matrix
    R-> Rotation Matrix
    t-> Translation vector
    
    P-> Projection Matrix.
    """
    
    if K.shape != (3,3):
        print('Check your Camera Matrix, it should have dimensions (3,3) and it has: ', K.shape)
    if R.shape != (3,3):
        print('Check your Rotatio Matrix, it should have dimensions (3,3) and it has: ', R.shape)
    if t.shape == (3,1):
        pass 
    else:
        t = squeezdim(t).reshape(-1,1)
    
    Rt = np.column_stack([R, t])
    P = K@Rt
    
    return P

***Create:*** <br>
    Camera matrix<br>
    Translation Vectors<br> 
    Rotation Matrix<br>
    Projection Matrix<br>
    Projected Points<br>
    List of points <br>
    List of Projected Matrices

In [33]:
K = np.matrix(np.array([[1000, 0, 300], [0, 1000, 200], [0, 0, 1]]))

t1 = np.zeros(3)
t2 = np.array([0.2, 2, 1])
t3 = np.array([0.9, 1.6, 1])
t4 = np.array([2, 1, .6])
t5 = np.array([1.1, -1, .9])
t6 = np.array([4, -1, -2])
t7 = np.array([-5, 5, 2])

R1 = np.matrix(np.eye(3))
R2 = np.matrix(Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix())
R3 = np.matrix(Rotation.from_euler('xyz', [-0.9, 0.2, 0.8]).as_matrix())
R4 = np.matrix(Rotation.from_euler('xyz', [0.5, -0.1, 0.5]).as_matrix())
R5 = np.matrix(Rotation.from_euler('xyz', [-0.3, 0.5, 0.4]).as_matrix())
R6 = np.matrix(Rotation.from_euler('xyz', [-0.9, 0.4, -0.5]).as_matrix())
R7 = np.matrix(Rotation.from_euler('xyz', [-0.4, -0.2, 0.9]).as_matrix())

In [34]:
P1 = projectMat(K, R1, t1)
P2 = projectMat(K, R2, t2)
P3 = projectMat(K, R3, t3)
P4 = projectMat(K, R4, t4)
P5 = projectMat(K, R5, t5)
P6 = projectMat(K, R6, t6)
P7 = projectMat(K, R7, t7)

In [35]:
Q1 = np.array([[5, 3, 1, 1]]).T

q1 = P1@Q1
q2 = P2@Q1
q3 = P3@Q1
q4 = P4@Q1
q5 = P5@Q1
q6 = P6@Q1
q7 = P7@Q1

q1n = q1/q1[-1]
q2n = q2/q2[-1]
q3n = q3/q3[-1]
q4n = q4/q4[-1]
q5n = q5/q5[-1]
q6n = q6/q6[-1]
q7n = q7/q7[-1]

qs = [q1, q2, q3, q4, q5]
Ps = [P1, P2, P3, P4, P5]

In [40]:
Ps

[matrix([[1000.,    0.,  300.,    0.],
         [   0., 1000.,  200.,    0.],
         [   0.,    0.,    1.,    0.]]),
 matrix([[ 7.55245320e+02, -5.94238880e+02,  4.08025317e+02,
           5.00000000e+02],
         [ 7.25424304e+02,  4.24382932e+02, -5.77631982e+02,
           2.20000000e+03],
         [ 4.79425539e-01,  5.65354208e-01,  6.71212166e-01,
           1.00000000e+00]]),
 matrix([[ 6.23218161e+02, -7.84653065e+02, -2.93118902e+02,
           1.20000000e+03],
         [ 6.63322863e+02,  1.67900200e+02,  7.56182718e+02,
           1.80000000e+03],
         [-1.98669331e-01, -7.67712524e-01,  6.09219154e-01,
           1.00000000e+00]]),
 matrix([[ 9.03148329e+02, -3.19629832e+02,  4.14921517e+02,
           2.18000000e+03],
         [ 4.96997091e+02,  8.42610639e+02, -2.88099293e+02,
           1.12000000e+03],
         [ 9.98334166e-02,  4.77030408e-01,  8.73198304e-01,
           6.00000000e-01]]),
 matrix([[ 6.64479405e+02, -5.80324427e+02,  5.58292647e+02,
           1.

##### Apply triangulation

In [162]:
Q, B = triangulate(qs, Ps)

Scale the Obtained point Q

In [163]:
Q = Q/Q[-1,-1]
Q

array([[5., 3., 1., 1.]])

Use more points

In [164]:
qs = [q1, q2, q3, q4, q5, q6, q7]
Ps = [P1, P2, P3, P4, P5, q6, q7]
Q, B = triangulate(qs, Ps)

In [165]:
Q = Q/Q[-1,-1]
Q

array([[5., 3., 1., 1.]])

Transform into homogenous

In [166]:
qs = [q1n, q2n, q3n, q4n, q5n, q6n ,q7n]
Ps = [P1, P2, P3, P4, P5, P6, P7]

In [167]:
Q, B = triangulate(qs, Ps)
Q = Q/Q[-1,-1]
Q

array([[5., 3., 1., 1.]])